## Recursos e bibliotecas para recuperar e manipular os dados

In [ ]:
#instala o sqlalchemy
!pip install sqlalchemy

In [13]:
#importa a criação de conexão
from sqlalchemy import create_engine 
import pandas as pd

## Configurando a conexão com nosso banco de dados

In [14]:
# Conectando com o Windows Authentication

try:
    Server = 'DESKTOP-9I30DAT'
    Database = 'ContosoRetailDW'
    Driver = 'ODBC Driver 17 for SQL Server'
    Database_con = f'mssql://@{Server}/{Database}?driver={Driver}'

    engine = create_engine(Database_con)
    conn = engine.connect()
    print("Conexão bem sucedida!")
except:
    print("Conexão falhou")

Conexão bem sucedida!


## Consulta para recuperação dos dados

In [15]:
# Consulta para pegarmos nossa base de dados do nosso banco
query = '''
        select 
                * 
        from vw_base_previsao 
        '''
df = pd.read_sql(query, conn)

#Mostrando as primeiras linhas da nossa base

display(df.head(5))

,ID da Venda,Data da Venda,Ano,Mês,Número do Mês,ID do Produto,Nome do Produto,Nome do Canal,Nome da Categoria,Nome da Subcategoria,Preço Unitário,Qtd. Vendida,Faturamento
0,1,02/01/2007,2007,January,1,956,A. Datum Point Shoot Digital Camera M500 Black,Store,Cameras and camcorders,Digital Cameras,198.0,8,1584.0
1,2,12/02/2007,2007,February,2,766,Contoso Battery charger - bike E200 Black,Reseller,Computers,Computers Accessories,19.9,4,79.6
2,3,24/01/2008,2008,January,1,1175,Fabrikam Budget Moviemaker 2/3'' 17mm E100 White,Store,Cameras and camcorders,Camcorders,410.0,9,3690.0
3,4,13/01/2008,2008,January,1,1429,The Phone Company Touch Screen Phones 4-Wire/O...,Online,Cell phones,Touch Screen Phones,289.0,8,2312.0
4,5,22/01/2008,2008,January,1,1133,"Fabrikam SLR Camera 35"" X358 Blue",Online,Cameras and camcorders,Digital SLR Cameras,436.2,24,10468.8


In [16]:
# encerrando a conexão
conn.close()

In [17]:
#Nossa base de dados
df

,ID da Venda,Data da Venda,Ano,Mês,Número do Mês,ID do Produto,Nome do Produto,Nome do Canal,Nome da Categoria,Nome da Subcategoria,Preço Unitário,Qtd. Vendida,Faturamento
0,1,02/01/2007,2007,January,1,956,A. Datum Point Shoot Digital Camera M500 Black,Store,Cameras and camcorders,Digital Cameras,198.00,8,1584.0
1,2,12/02/2007,2007,February,2,766,Contoso Battery charger - bike E200 Black,Reseller,Computers,Computers Accessories,19.90,4,79.6
2,3,24/01/2008,2008,January,1,1175,Fabrikam Budget Moviemaker 2/3'' 17mm E100 White,Store,Cameras and camcorders,Camcorders,410.00,9,3690.0
3,4,13/01/2008,2008,January,1,1429,The Phone Company Touch Screen Phones 4-Wire/O...,Online,Cell phones,Touch Screen Phones,289.00,8,2312.0
4,5,22/01/2008,2008,January,1,1133,"Fabrikam SLR Camera 35"" X358 Blue",Online,Cameras and camcorders,Digital SLR Cameras,436.20,24,10468.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,199996,22/12/2008,2008,December,12,972,A. Datum Compact Digital Camera M200 Pink,Store,Cameras and camcorders,Digital Cameras,129.00,8,1032.0
199996,199997,18/03/2008,2008,March,3,1064,A. Datum SLR Camera X140 Pink,Online,Cameras and camcorders,Digital SLR Cameras,627.00,18,11286.0
199997,199998,08/07/2008,2008,July,7,2110,Contoso Water Heater 7.2GPM X1800 Red,Catalog,Home Appliances,Water Heaters,1475.00,12,17700.0
199998,199999,13/07/2008,2008,July,7,716,Proseware High Speed Laser M2000 White,Catalog,Computers,"Printers, Scanners & Fax",209.00,24,5016.0


Neste caso, nossa base ainda não está no formato ideal para realizarmos a modelagem. Precisaremos agrupar algumas colunas por mês e ano

In [ ]:
#Criando uma nova coluna Ano/mês na nossa base
df[Ano/mês] = 

## Análise Exploratória

## Modelagem

### Bibliotecas utilizadas para a análise e modelagem

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics